# Goal-oriented self-improvement for Automated Theorem Proving

## Method
We are basically thinking of extending minimo by introducing a goal i.e. a set of problems the model should eventually solve. We do this by adding loss-term forcing the model to sample conjectures from our goal set. We control this ‘forcing’ using a hyperparameter `alpha`. A higher value gives the `progess_loss` more value, thus pushes the model towards the goal stronger.

In [8]:
# lets move to the parent directory so it is easier to run the scripts
import os

if os.getcwd().split('/')[-1] == 'experiments':
    os.chdir('../')

!ls


count_errors.py   README.md		   slurm-1283594.out  slurm-1284247.out
Dockerfile	  redis_hostname_port.txt  slurm-1283595.out  slurm-1284248.out
environment	  setup.sh		   slurm-1283596.out  slurm-1284249.out
experiments	  slurm-1283480.out	   slurm-1283597.out  slurm-1284250.out
FAQ.md		  slurm-1283487.out	   slurm-1283598.out  slurm-1284463.out
goals		  slurm-1283573.out	   slurm-1283599.out  slurm-1284464.out
install_redis.sh  slurm-1283582.out	   slurm-1284225.out  slurm-1284465.out
launch		  slurm-1283583.out	   slurm-1284240.out  slurm-1284466.out
learning	  slurm-1283584.out	   slurm-1284241.out  slurm-1284467.out
LICENSE		  slurm-1283585.out	   slurm-1284242.out  slurm-1284468.out
logs		  slurm-1283586.out	   slurm-1284243.out  slurm-1284469.out
ls		  slurm-1283587.out	   slurm-1284244.out  tutorial.md
outputs		  slurm-1283588.out	   slurm-1284245.out  wandb
pyproject.toml	  slurm-1283593.out	   slurm-1284246.out


In [4]:
# Let's start some workers so our tasks get executed in parallel. 
# @franz you would need to start them on the same node
!sbatch --job-name=redis --cpus-per-task=10 --mem=250G --time=20:00:00 --wrap="./launch/start_redis.sh -v"

!sbatch --job-name=worker1 --cpus-per-task=3 --gres=gpu:1 --mem=20G --time=20:00:00 --wrap="./launch/start_worker.sh"
!sbatch --job-name=worker2 --cpus-per-task=3 --gres=gpu:1 --mem=20G --time=20:00:00 --wrap="./launch/start_worker.sh"
# !sbatch --job-name=worker3 --cpus-per-task=3 --gres=gpu:1 --mem=20G --nodelist=node[6] --time=20:00:00 --wrap="./launch/start_worker.sh"
# !sbatch --job-name=worker4 --cpus-per-task=3 --gres=gpu:1 --mem=50G --nodelist=node[6] --time=20:00:00 --wrap="./launch/start_worker.sh"
# !sbatch --job-name=worker5 --cpus-per-task=3 --gres=gpu:1 --mem=50G --nodelist=node[6] --time=20:00:00 --wrap="./launch/start_worker.sh"


Submitted batch job 1276568
Submitted batch job 1276569
Submitted batch job 1276570


## Experiments

For the now we use a very simple goal set. It consists of a single goal which is proving the theorem: 

`a is a natural number: (0 + a) = a`

Or in peano:

`[('a0 : nat) -> (= (+ z 'a0) 'a0)]`

### 1. Overfit on the goal set
After implementing the goal-conditioning, let's run experiments with `alpha=0` and `alpha=1`. We expect the `progress_loss` to go down i.e. the model overfits to the `final_goal` set. However, the actual `train_loss` shouldn’t go down. 

In [20]:
# start a job with alpha=1 to overfit on the goals
!sbatch --job-name=train_alpha_1 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=1:00:00 --wrap="./launch/run_bootstrap_distributed.sh agent.max_mcts_nodes=100 agent.policy.alpha=1 agent.policy.total_iterations=2"

# start a job with alpha=0 as a baseline
!sbatch --job-name=train_alpha_0 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=1:00:00 --wrap="./launch/run_bootstrap_distributed.sh agent.max_mcts_nodes=100 agent.policy.alpha=0 agent.policy.total_iterations=2"

Submitted batch job 1275940
Submitted batch job 1275941


Works! The ``training_loss`` diverges for `alpha=1`. The `progress_loss` struggles to go below a certain threshold for `alpha=0`. As per our intuition, the former overfits to sampling the final theorem and can't solve any conjectures in the second iteration. We need something better for alpha.

### 2. Try different values for alpha

We try different schedules for alpha in the hope that the training loss and progress loss both converge. 

In [3]:
# TODO this still runs on single GPU
!sbatch --job-name=train_alpha_0_8 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.8"

!sbatch --job-name=train_alpha_0_6 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.6"

!sbatch --job-name=train_alpha_0_4 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.4"

!sbatch --job-name=train_alpha_0_2 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.2"

!sbatch --job-name=train_alpha_0_3e_4 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=3e-4"

!sbatch --job-name=train_alpha_0_1e_3 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=24:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1e-3"

Submitted batch job 1275099


This works as well. We also observe a correlation between alpha values and the ratio of conjectures that the model is able to prove per iteration. The higher the alpha value, the fewer problems the model can solve. What if we could let the model explore problems for itself for a while and at a later stage push it towards our goal?

### 3 Alpha Schedules

We implement fancy schedules to 'warm-up' alpha over several iterations. The options are 

`alpha_schedule = [ constant | linear | quadratic | cubic | cos ]`

#### 3.1 Warm up alpha to 1.0

As a first step we warm up alpha to 1.0. 

In [6]:
# TODO this still runs on single GPU
!sbatch --job-name=train_alpha_lin --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1 alpha_schedule=linear"

!sbatch --job-name=train_alpha_quad --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1 alpha_schedule=quadratic"

!sbatch --job-name=train_alpha_cubic --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1 alpha_schedule=cubic"

!sbatch --job-name=train_alpha_cos --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1 alpha_schedule=cos"

Submitted batch job 1274969


It is apparent that high values of alpha just overpower the actual loss. Ideally we wan't to keep alpha much smaller. Let's try just warming it up to lower values. 

#### 3.2 Warm up alpha to lower values


In [3]:
# TODO this still runs on single GPU
!sbatch --job-name=train_alpha_cubic --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.2 alpha_schedule=cubic"

!sbatch --job-name=train_alpha_cubic --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.3 alpha_schedule=cubic"

!sbatch --job-name=train_alpha_cubic --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.4 alpha_schedule=cubic"

Submitted batch job 1274557
Submitted batch job 1274558
Submitted batch job 1274559


### 4 Use the ratio of solved conjectures 

Now comes the actually interesting part. We implement a more principled schedule.
Let's use the ratio of solved conjectures to total sampled conjectures to directly control alpha. 

In [4]:
# TODO this still runs on single GPU
!sbatch --job-name=train_alpha_ratio --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1 alpha_schedule=ratio"

!sbatch --job-name=train_alpha_ratio --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0.2 alpha_schedule=ratio"

Submitted batch job 1274560
Submitted batch job 1274561


In [16]:
# start a job with alpha schedule ratio and max_alpha=1
!sbatch --job-name=train_ratio --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=1 agent.policy.alpha_schedule=ratio goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"


# start a job with alpha schedule ratio and max_alpha=5e-3
!sbatch --job-name=train_ratio_small --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=5e-3 agent.policy.alpha_schedule=ratio goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

# start a job with alpha=0 as a baseline
!sbatch --job-name=train_vanilla --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=0 goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

Submitted batch job 1276645
Submitted batch job 1276646
Submitted batch job 1276647


In [3]:
# constants

# # start a job with alpha schedule const and max_alpha=0.1e-2
# !sbatch --job-name=train_constant_1e_2 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=1e-2 agent.policy.alpha_schedule=constant goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

# # start a job with alpha schedule const and max_alpha=0.1e-3
# !sbatch --job-name=train_constant_1e_3 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=1e-3 agent.policy.alpha_schedule=constant goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

# start a job with alpha schedule const and max_alpha=0.1e-4
# !sbatch --job-name=train_constant_1e_4 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=1e-4 agent.policy.alpha_schedule=constant goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

# start a job with alpha schedule const and max_alpha=0.1e-5
!sbatch --job-name=train_constant_1e_4 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=1e-5 agent.policy.alpha_schedule=constant goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

# start a job with alpha schedule const and max_alpha=0.1e-6
!sbatch --job-name=train_constant_1e_4 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=1e-6 agent.policy.alpha_schedule=constant goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

# ratio
# start a job with alpha schedule ratio and max_alpha=0.1e-2
# !sbatch --job-name=train_ratio1e_2 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=1e-2 agent.policy.alpha_schedule=ratio goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

# # start a job with alpha schedule ratio and max_alpha=0.1e-3
# !sbatch --job-name=train_ratio1e_3 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=1e-3 agent.policy.alpha_schedule=ratio goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

# # start a job with alpha schedule ratio and max_alpha=0.1e-4
# !sbatch --job-name=train_ratio1e_4 --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=1e-4 agent.policy.alpha_schedule=ratio goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

Submitted batch job 1276686
Submitted batch job 1276687


In [2]:
# start a job with alpha schedule ratio and max_alpha=1
!sbatch --job-name=train_ratio_nl --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --nodelist=node[5] --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=1 agent.policy.alpha_schedule=ratio agent.policy.normalize_loss=true goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

# start a job with alpha schedule ratio and max_alpha=5e-3
!sbatch --job-name=train_ratio_small_nl --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --nodelist=node[5] --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add agent.policy.alpha=5e-3 agent.policy.alpha_schedule=ratio agent.policy.normalize_loss=true goals=nat-add-synthetic n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.total_iterations=1000 agent.policy.train_iterations=500"

Submitted batch job 1276208
Submitted batch job 1276209


#### 4.1 Increase max iters

In [3]:
!sbatch --job-name=train_alpha_long --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=20:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=0 iterations=30"

!sbatch --job-name=train_alpha_long_ratio --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=20:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1 alpha_schedule=ratio iterations=30"

!sbatch --job-name=train_alpha_long_cos --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=20:00:00 --wrap="python learning/bootstrap.py theory=nat-add alpha=1 alpha_schedule=cubic iterations=30"

Submitted batch job 1275039
Submitted batch job 1275040
Submitted batch job 1275041


## 5. New config and `mu`

over the weekend we had a few changes. 
* we renamed`alpha` to `mu`. 
* we omit having a fancy `alpha_schedule` and work with a constant `mu` 
* we set `mu` to zero if the ratio of solved problems goes under a certain `threshold`
* we multiply `mu` by the ratio of difficulty-problem pairs to the total training set size. The thought behind this is that we only want to influence the conjecturing. Not the 'solving'.
* A couple of bugfixes

As a first run let's try out different values of mu and see what happens. For iteration speed we go down to much fewer conjectures and mcts steps. 


In [3]:
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=1 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.1 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.05 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.005 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-synthetic"

Submitted batch job 1278167
Submitted batch job 1278168


ok these runs seem to do very different stuff even though in the beginning (when the prove ratio is below threshold) they should behave very similarly? Is this just randomness? Let's try a single run with different seeds

### 

### 5.2 Different seed experiment

In [ ]:
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=0"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=10"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=15"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=200"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=69"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=420"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=15 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=150 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=80085"

Ok so the variance is too high in these experiments (std=0.0625). Let's check if the variance is lower when we increase the number of conjectures

In [3]:
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=200 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=2000 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=0"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=200 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=2000 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=10"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=200 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=2000 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=15"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=200 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=2000 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=200"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=200 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=2000 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=69"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=200 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=2000 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=420"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=200 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 agent.policy.train_iterations=2000 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic seed=80085"

Submitted batch job 1278290
Submitted batch job 1278291
Submitted batch job 1278292
Submitted batch job 1278293
Submitted batch job 1278294
Submitted batch job 1278295
Submitted batch job 1278296


yes variance is much lower (std=0.0025). ok seems like we need to go back to big runs. try the config we had earlier that we used to generate the synth nat-add goal. When we generated the goal, it took the vanilla run 6 iterations. let's hope it finds it again

In [4]:

!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=1 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.1 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.05 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.005 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-synthetic"

Submitted batch job 1278307
Submitted batch job 1278308
Submitted batch job 1278309
Submitted batch job 1278310
Submitted batch job 1278311
Submitted batch job 1278312
Submitted batch job 1278313


The proved ratio is just way too low. Can we artificially get it up by increasing the percentile of hard (to 50)? (And easy to 70) Since we are already adding 'difficulty' through our goal conditioning, we might be breaking the equilibrium there.

In [7]:
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.005 goals=nat-add-synthetic"

Submitted batch job 1278322


alright so proved ratio went up! The val loss is going down as well! Now lets sweep different values for mu 

In [3]:
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.1 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.05 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.01 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.005 goals=nat-add-synthetic"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-synthetic"

Submitted batch job 1278364
Submitted batch job 1278365
Submitted batch job 1278366
Submitted batch job 1278367
Submitted batch job 1278368


okay so it seems like `mu=0.001` is the most promising value out of these runs. Just to confirm that we were not 'lucky', let's try it with different seeds. We expect `mu=0.001` to consistently have lower validation loss as minimo. 

In [3]:
# mu=0.001 with different seeds
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-synthetic seed=0"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-synthetic seed=42"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-synthetic seed=15"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-synthetic seed=200"

# vanilla with different seeds
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0 goals=nat-add-synthetic seed=0"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0 goals=nat-add-synthetic seed=42"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0 goals=nat-add-synthetic seed=15"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0 goals=nat-add-synthetic seed=200"

Submitted batch job 1278747
Submitted batch job 1278748
Submitted batch job 1278749
Submitted batch job 1278750
Submitted batch job 1278751
Submitted batch job 1278752
Submitted batch job 1278753
Submitted batch job 1278754


Okay so the variance between these runs is immense. However, what we can see is that for the vanilla runs, it is completely random if and when the val loss goes down. It often even goes back up. This makes sense because the directions the model improves into is random. 
For us 1 out of 3 runs actually got up to a proved_ratio that was above the threshold value of 0.3. Need to investigate why! However the run that did get above the threshold, also eventually had the lowest val_loss out of all the runs.

up until now we have seen that our method works towards the goal while minimo doesn't necessarily. We saw this through the val loss. Now let's use a slightly simpler goal with fewer mcts steps and see if we can see the same results while actually eventually solving the goal. 

In [4]:
# baseline run
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=100 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0 goals=nat-add-easy"
# baseline run with different seed
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=100 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0 goals=nat-add-easy seed=42"

# ours with different seeds
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=100 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-easy"
!sbatch --job-name=train_minimo --nodelist=node[6] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=100 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-easy seed=42"

Submitted batch job 1278978
Submitted batch job 1278979
Submitted batch job 1278980
Submitted batch job 1278981


and run them with even fewer mcts steps

In [5]:
# baseline run
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0 goals=nat-add-easy"
# baseline run with different seed
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0 goals=nat-add-easy seed=42"

# ours with different seeds
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-easy"
!sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=5:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=50 agent.policy.batch_size=10000 difficulty_buckets=[{hard:50},{easy:70},{triv:100}] agent.policy.train_iterations=500 agent.policy.total_iterations=1000 agent.policy.mu=0.001 goals=nat-add-easy seed=42"

Submitted batch job 1278982
Submitted batch job 1278983
Submitted batch job 1278984
Submitted batch job 1278985


In [9]:
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1 --nodelist=node[5] --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0.001 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-3 agent.policy.threshold=0.2 agent.policy.mu_warmup_steps=1000 job.name=long_warmup_normal_mu-1"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1 --nodelist=node[5] --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0.001 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-3 agent.policy.threshold=0.2 agent.policy.mu_warmup_steps=1000 job.name=long_warmup_normal_mu-2"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1 --nodelist=node[5] --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0.001 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-3 agent.policy.threshold=0.2 agent.policy.mu_warmup_steps=1000 job.name=long_warmup_normal_mu-3"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1 --nodelist=node[6] --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0.001 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-3 agent.policy.threshold=0.2 agent.policy.mu_warmup_steps=1000 job.name=long_warmup_normal_mu-4"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1 --nodelist=node[6] --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0.001 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-3 agent.policy.threshold=0.2 agent.policy.mu_warmup_steps=1000 job.name=long_warmup_normal_mu-5"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1 --nodelist=node[6] --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0.001 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-3 agent.policy.threshold=0.2 agent.policy.mu_warmup_steps=1000 job.name=long_warmup_normal_mu-6"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1 --nodelist=node[6] --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0.001 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-3 agent.policy.threshold=0.2 agent.policy.mu_warmup_steps=1000 job.name=long_warmup_normal_mu-7"

# !sbatch --job-name=train_minimo --nodelist=node[5] --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0.002 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-zeros agent.policy.threshold=0.2 agent.policy.mu_warmup_steps=2500"

Submitted batch job 1284510
Submitted batch job 1284511
Submitted batch job 1284512
Submitted batch job 1284513
Submitted batch job 1284514
Submitted batch job 1284515
Submitted batch job 1284516


In [5]:
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-4 agent.policy.threshold=0 agent.policy.mu_warmup_steps=2500 job.name=scaling-goals-vanilla-1"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-4 agent.policy.threshold=0 agent.policy.mu_warmup_steps=2500 job.name=scaling-goals-vanilla-2"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-4 agent.policy.threshold=0 agent.policy.mu_warmup_steps=2500 job.name=scaling-goals-vanilla-3"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-4 agent.policy.threshold=0 agent.policy.mu_warmup_steps=2500 job.name=scaling-goals-vanilla-4"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-4 agent.policy.threshold=0 agent.policy.mu_warmup_steps=2500 job.name=scaling-goals-vanilla-5"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-4 agent.policy.threshold=0 agent.policy.mu_warmup_steps=2500 job.name=scaling-goals-vanilla-6"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-4 agent.policy.threshold=0 agent.policy.mu_warmup_steps=2500 job.name=scaling-goals-vanilla-7"
!sbatch --job-name=train_minimo --cpus-per-task=4 --mem=50G --gres=gpu:1,VRAM=12G --time=10:00:00 --wrap="python learning/bootstrap.py theory=nat-add n_conjectures=50 agent.max_mcts_nodes=200 agent.expansions=500 agent.policy.batch_size=10000 agent.policy.train_iterations=500 agent.policy.total_iterations=50 agent.policy.mu=0 difficulty_buckets=[{hard:20},{easy:50},{triv:100}] goals=nat-add-scaling-4 agent.policy.threshold=0 agent.policy.mu_warmup_steps=2500 job.name=scaling-goals-vanilla-0"


Submitted batch job 1284247
Submitted batch job 1284248
Submitted batch job 1284249
Submitted batch job 1284250
Submitted batch job 1284251
Submitted batch job 1284252
Submitted batch job 1284253
Submitted batch job 1284254
